In [126]:
import numpy as np
import cv2 as cv


# initial values
brightness = 10
contrast_value = 10
angle_value = 0.0
scale = 1.0

marker_size = 2

## Upload image

In [127]:
img = cv.imread('Task-9.jpg')
harris_img = img.copy()
shi_tomachi_img = img.copy()
transformed_img = img

In [128]:
# For affine
height, width = img.shape[:2]
center = (width / 2, height / 2)
# For perpective
point_list = list()
point_list.append((0, 0))
point_list.append((img.shape[1], 0))
point_list.append((img.shape[1], img.shape[0]))
point_list.append((0, img.shape[0]))
point_for_change = 0
# For Harris
# block_size - It is the size of neighbourhood considered for corner detection.
# k_size - Aperture parameter of the Sobel derivative used.
# k_p - Harris detector free parameter in the equation.
block_size = 2
k_size = 3
k_p = 0.001
# For Shi_Tomachi
corners_count = 80
corners_quality = 0.05
minimal_distance = 5
show_gray_shi = 0

# Image transformations

In [129]:
def update():
	global transformed_img
	transformed_img = img.copy()
	change_brightness(brightness)
	change_contrast(contrast_value)
	rotation_matrix = cv.getRotationMatrix2D(center, angle_value, scale)
	transformed_img = cv.warpAffine(transformed_img, rotation_matrix, (width, height))
	# For perspective transformation, you need a 3x3 transformation matrix. Straight lines will remain straight even after the transformation. To find this transformation matrix, you need 4 points on the input image and corresponding points on the output image. Among these 4 points, 3 of them should not be collinear. Then the transformation matrix can be found by the function cv.getPerspectiveTransform. Then apply cv.warpPerspective with this 3x3 transformation matrix.
	points_for_transformation = np.float32([[0, 0], [img.shape[1], 0], [img.shape[1], img.shape[0]], [0, img.shape[0]]])
	matrix = cv.getPerspectiveTransform(
		points_for_transformation,
		np.float32(point_list))
	transformed_img = cv.warpPerspective(transformed_img, matrix, (img.shape[1], img.shape[0]), cv.INTER_CUBIC,
										 borderMode=cv.BORDER_CONSTANT, borderValue=(0, 0, 0))
	# cv.imshow('Pic', transformed_img)
	update_Harris()
	update_Shi_tomachi()


def change_brightness(brightness):
	global transformed_img
	if brightness != 0:
		transformed_img = cv.addWeighted(transformed_img, (255 - brightness) / 255, transformed_img, 0, brightness)


def change_contrast(contrast):
	global transformed_img
	if contrast != 0:
		f = 131 * (contrast + 127) / (127 * (131 - contrast))
		transformed_img = cv.addWeighted(transformed_img, f, transformed_img, 0, 127 * (1 - f))


def change_brightness_value(val):
	global brightness
	brightness = val
	update()


def change_contrast_value(val):
	global contrast_value
	contrast_value = val
	update()


def change_rotate_angle(val):
	global angle_value
	angle_value = val
	update()


def change_rotate_scale(val):
	global scale
	if val > 0:
		scale = val / 10.0
		update()


def change_point_x_val(val):
	global point_list
	_, old_y = point_list[point_for_change]
	point_list[point_for_change] = (val, old_y)
	update()


def change_point_y_val(val):
	global point_list
	old_x, _ = point_list[point_for_change]
	point_list[point_for_change] = (old_x, val)
	update()


In [130]:
def update_Harris():
	gray = cv.cvtColor(transformed_img, cv.COLOR_BGR2GRAY)
	corners = cv.cornerHarris(gray, block_size, k_size, k_p)
	harris_img = transformed_img.copy()
	harris_img[corners > 0.01 * corners.max()] = [0, 255, 0]
	cv.imshow('Harris', harris_img)


def change_block_size(val):
	global block_size
	block_size = val
	update_Harris()


def change_k_size(val):
	global k_size
	# четное ядро почему-то не отличается по эффекту от нечетного
	if val % 2 == 0:
		val += 1
	k_size = val
	update_Harris()


def change_k_p(val):
	global k_p
	if val > 0:
		k_p = val / 100
		update_Harris()

In [131]:
def draw_markers_shi(img_, corners, color=(0, 255, 0)):
	corners_int = np.int0(corners)
	for corner in corners_int:
		x, y = corner.ravel()
		cv.drawMarker(img_, (x, y), color, cv.MARKER_CROSS, marker_size)


def update_Shi_tomachi():
	gray = cv.cvtColor(transformed_img, cv.COLOR_BGR2GRAY)
	corners = cv.goodFeaturesToTrack(gray, corners_count, corners_quality, minimal_distance)
	shi_tomachi_img = transformed_img.copy()
	corners_int = np.int0(corners)
	for corner in corners_int:
		x, y = corner.ravel()
		cv.drawMarker(shi_tomachi_img, (x, y), (0, 255, 0), cv.MARKER_CROSS, marker_size)
	cv.imshow('Shi-Tomachi', shi_tomachi_img)


def change_corners_count(val):
	global corners_count
	if val > 0:
		corners_count = val
		update_Shi_tomachi()


def change_corners_quality(val):
	global corners_quality
	if val > 0:
		corners_quality = val / 100.0
		update_Shi_tomachi()


def change_minimal_distance(val):
	global minimal_distance
	minimal_distance = val
	update_Shi_tomachi()


def change_marker_size(val):
	global marker_size
	marker_size = val
	update_Shi_tomachi()

In [132]:
# Do some things with begin pic
cv.namedWindow('Control')
cv.createTrackbar('Brightness', 'Control', 0, 255, change_brightness_value)
cv.createTrackbar('Contrast', 'Control', 0, 128, change_contrast_value)
cv.createTrackbar('Angle', 'Control', 0, 360, change_rotate_angle)
cv.createTrackbar('Scale', 'Control', 10, 360, change_rotate_scale)

cv.createTrackbar('X', 'Control', 0, img.shape[1] - 1, change_point_x_val)
cv.createTrackbar('Y', 'Control', 0, img.shape[0] - 1, change_point_y_val)

#Harris corners
cv.imshow('Harris', harris_img)
cv.createTrackbar('Block size', 'Harris', 2, 50, change_block_size)
cv.createTrackbar('Kernel size', 'Harris', 3, 31, change_k_size)
cv.createTrackbar('K', 'Harris', 10, 100, change_k_p)

#Shi-Tomachi corners
cv.imshow('Shi-Tomachi', shi_tomachi_img)
cv.createTrackbar('Corners count', 'Shi-Tomachi', 80, 300, change_corners_count)
cv.createTrackbar('Quality', 'Shi-Tomachi', 5, 100, change_corners_quality)
cv.createTrackbar('Min distance', 'Shi-Tomachi', 5, 100, change_minimal_distance)
cv.createTrackbar('Marker size', 'Shi-Tomachi', 2, 30, change_marker_size)

cv.waitKey(0)
cv.destroyAllWindows()